In [1]:
import pandas, numpy as np,re
import xml, itertools, re, xml.etree.ElementTree as ET
from html.parser import HTMLParser
import datetime, tqdm
pars =HTMLParser()

def prettyPrintXML(s):
    r = xml.dom.minidom.parseString(s) #r.RegistroXML)
    print(r.toprettyxml())
    
#from datetime import datetime as dt
#import locale
#locale.setlocale(locale.LC_ALL, 'es_es')
#spanishDateToPandas = lambda s: pandas.to_datetime(dt.strptime(s, '%d %b %Y %H:%M:%S'))

# Data reading and basic cleaning

- Find candidates of repeated pateints
- Find the used codes for diagnosis and procedures

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)

#str2Date = lambda s:datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f') if isinstance(s, str) \
#           else datetime.datetime(1, 1, 1)
registros.FechaInicioAtencion = registros.FechaInicioAtencion.map(pandas.to_datetime) #registros.FechaInicioAtencion.map(str2Date)
procedimientos.FechaDescripcion  = procedimientos.FechaDescripcion.map(pandas.to_datetime)
from html.parser import HTMLParser
pars =HTMLParser()
#pars.unescape

C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,27,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
procedureIDFound = set()
for _, p in procedimientos.iterrows():
    et = xml.etree.ElementTree.fromstring(p.XmlDescripcion)
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', p.XmlDescripcion)
    for ss in s:
        procedureIDFound.add(ss)
procedureIDFound = list(sorted(procedureIDFound))

In [8]:
procedimientosDesc.loc[procedureIDFound].to_excel('procedimientosFiltered.xlsx')
procedimientosDesc.loc[procedureIDFound].to_csv('procedimientosFiltered.csv')

In [5]:
unique = registros.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
for u in unique:
    if u.upper() not in diagnosis.index:
        print(u)

0800


In [9]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

# Select the cases by procedure

- Based on the procedure, select the history register associated.
- TODO: Check that diagnosis and procedure are compatible

In [10]:
registersByPatient = registros.groupby('NumeroHistoria')

In [21]:
####
## Old code, before I discovered the link between case and operations
####
#getprocID = lambda s:  re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', s)[0]
#getprocDate = lambda s:  re.findall('<fechaCirugia>(.*)</fechaCirugia>', s)[0]
#
#def associateCaseAndHistory(interventionId, intervention, registersByPatient, 
#                            nDaysThreshold = 3,  nDaysUpperThreshold = 0):
#    pId = intervention.IdPaciente
#    df = registersByPatient.get_group(pId)
#    #interventionDate = spanishDateToPandas(getprocDate(intervention.XmlDescripcion))
#    interventionDate = pandas.to_datetime(intervention.FechaRegistro.replace('T', ''))
#    df = df.loc[df.FechaInicioAtencion.between(interventionDate - pandas.Timedelta(nDaysThreshold, 'D'),
#                                      interventionDate +  pandas.Timedelta(nDaysUpperThreshold, 'D'))]
#    #Do not count Triage in the cases ID
#    caseIds =  [s for s in df.Caso.unique() if not s.startswith('TRI')]
#    if len(caseIds) == 0:
#        print('Error in intervention %d found %d. Code = %s. Patient = %s' % (interventionId, df.Caso.nunique(),
#                                                                getprocID(intervention.XmlDescripcion), intervention.IdPaciente ))
#        print(intervention.FechaDescripcion, intervention.FechaRegistro, interventionDate)
#        return False
#    else:
#        return df.loc[df.FechaInicioAtencion == df.FechaInicioAtencion.max()].Caso.values[0]
#Identify borns and abortions using the proc ID
#interventionToCase = {}
#for k, i in tqdm.tqdm_notebook(procedimientos.iterrows()):
#    interventionToCase[k] = (associateCaseAndHistory(k, i, registersByPatient), i.IdPaciente)


In [141]:
interventionToCase = {}
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcion = int(et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo'))
        interventionToCase[idDescripcion] = (r.Caso, r.NumeroHistoria, i)

In [152]:
errors = []
for i, (_, k, k2) in interventionToCase.items():
    if procedimientos.loc[i].IdPaciente != k:
        print('ERROR: Patient name does not coincides', i, k2)
        errors.append(i)
for i in errors:
    del interventionToCase[i]

ERROR: Patient name does not coincides 61066 1356698


# Parse information from cases
Now we need to parse the information from the cases

In [154]:
def getStateDataFromCase(case):
    """
    Get some information from the case
    """
    data = {}
    r = ET.fromstring(case.RegistroXML)
    try:
        data['FC']=r.find('.//row[@NombreCampo="FrecuenciaCardiaca"]').get('ValorCampo')
        data['FR']=r.find('.//row[@NombreCampo="FrecuenciaRespiratoria"]').get('ValorCampo')
        data['Presion']=r.find('.//row[@NombreCampo="Presion"]').get('ValorCampo')
        data['Temperatura']=r.find('.//row[@NombreCampo="Temperatura"]').get('ValorCampo')
        data['Talla']=r.find('.//row[@NombreCampo="Talla"]').get('ValorCampo')
        data['Peso']=r.find('.//row[@NombreCampo="Peso"]').get('ValorCampo')
        return data
    except:
        return {}

In [155]:
floatParse = '[0-9]*\.?[0-9]+'
compareGPCA = lambda s1, s2: all([i == j or not i or not j for i, j in zip(s1, s2)])
allEqual = lambda s: all([compareGPCA(i, j) for i, j in  itertools.combinations(s, r = 2)])
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGPAC = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGPAC += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGPAC += '|'
patternGPAC += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGPAC += '|' # C and A can be missing, or in other order
patternGPAC += ')'
searchGPAC = re.compile(patternGPAC, flags=0)

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
noRecuerda = ['NO', '\?']
separadorFecha = '(?:[\.\\/-]|DE)?'
date = '(([0-9]+)'+ sep + separadorFecha + sep + '([0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '([0-9]+)'

searchFUM = re.compile('FUM'+ sep + '(?::|.)?'+ sep + date + '|%s)' % ('|'.join(noRecuerda)), flags = re.IGNORECASE)
errors = []
def parseGPCA_and_fum(s):
    """
    Gets the GPCA and FUM from the Triage or epicrisis
    
    TODO: actually, registro del recien nacido has it as a field.
    """
    r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',ET.fromstring(s).findall('row')))
    if len(r) == 0:
         return {}
    text = pars.unescape(r[0].attrib['ValorCampo'])
    
    allowedStarts = ['7-&gt;', '-', '- ANTECEDENTES']
    line = re.findall('^%s(?:%s)?%s' % (sep, '|'.join(allowedStarts), sep) + 'G' + sep + '[0-9]+.*$', text, re.M)
    if len(line) != 1:
        #If it doesn't find it, search brutally
        search = searchGPAC.findall(s)
        searchCleaned = [(s[0], s[1], s[2] + s[-1], s[3] + s[-2]) for s in search]
        if search and allEqual(search):
            GPCA_OK = True
            parsedGPCA = search[0]
        else:
            GPCA_OK = False
            parsedGPCA = []

    else:
        f = lambda s: s[0] if s else ''
        G = f(re.findall('G' + sep + '([0-9]+)', line[0]))
        A = f(re.findall('A' + sep + '([0-9]+)', line[0]))
        C = f(re.findall('C' + sep + '([0-9]+)', line[0]))
        P = f(re.findall('P' + sep + '([0-9]+)', line[0])) 
        parsedGPCA = [G, P, A, C]
        GPCA_OK = True
    #Prob athere is a better way...
    parsedFUM = searchFUM.findall(s)
    return {'fum' : len(parsedFUM) > 0,
            'fum_OK' : len(parsedFUM) ,
            'GPCA_OK' : GPCA_OK,
            'fum_Data' : parsedFUM,
            'parsedGPCA' : parsedGPCA}

In [156]:
from IPython.display import display

def getBackgroundRegister(casoID, registrosByCase):
    """
    Gets a register that includes the personal background in order Epicrisis / Triage / Ingreso a urgencias
    """
    #
    epicrisis = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Epicrisis', case = False) == True)]
    if epicrisis is not None and len(epicrisis):
        return epicrisis
    
    triage = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Triage', case = False) == True)]
    if triage is not None and len(triage):
        return triage

    urgencias = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Ingreso de Urgencias', case = False) == True)]
    if urgencias is not None:
        return urgencias


In [159]:
errorsBackgroundRegister = []
backgroundByCase = {}
dataGPAC_fum = {}
for k, (case, pId, _) in tqdm.tqdm_notebook(interventionToCase.items()):
    if case is False:
        continue
        
    backgroundByCase[case] = getBackgroundRegister(case, registersByPatient.get_group(pId))
    if len(backgroundByCase[case]) == 0:
        errorsBackgroundRegister.append(case)
    else: 
        dataGPAC_fum[case] = parseGPCA_and_fum(backgroundByCase[case].iloc[0].RegistroXML)

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.


In [160]:
dataGPAC_fumDF

,fum,fum_OK,GPCA_OK,fum_Data,parsedGPCA
AD40216,True,1,True,"[(7 JUNIO 2014, 7, JUNIO, 2014)]","(5, 3, 1, 3, , )"
AD152124,True,1,True,"[(3/12/2015, 3, 12, 2015)]","[3, 3, 0, 0]"
AD169414,True,1,True,"[(3-03-2016, 3, 03, 2016)]","[5, 3, 1, 0]"
AD182404,True,1,True,"[(NO, , , )]","[2, 1, 0, ]"
AD184378,True,3,True,"[(NO, , , ), (NO, , , ), (NO, , , )]","[1, 0, 0, 0]"
...,...,...,...,...,...
AD368548,True,2,True,"[(NO, , , ), (NO, , , )]","[1, , 1, ]"
AD368000,True,4,True,"[(9/04/19, 9, 04, 19), (40.2, 4, 0, 2), (9/04/...","[1, 0, 0, 0]"
AD368822,True,4,True,"[(3/05/19, 3, 05, 19), (36.3, 3, 6, 3), (3/05/...","[3, 2, 0, 1]"
AD368769,True,2,True,"[(2/03/19, 2, 03, 19), (2/03/19, 2, 03, 19)]","[3, 2, 0, 0]"


In [86]:
np.sum(dataGPAC_fumDF.GPCA_OK == 0), np.sum(dataGPAC_fumDF.GPCA_OK == 0)

,fum,fum_OK,GPCA_OK,fum_Data,parsedGPCA
AD273164,True,1,False,"[(NO, , , )]",[]
AD284225,False,0,False,[],[]
AD287970,True,1,False,"[(8-07-17, 8, 07, 17)]",[]
AD298467,False,0,False,[],[]
AD301211,False,0,False,[],[]
...,...,...,...,...,...
AD366549,True,1,False,"[(7/ABRIL/2019, 7, ABRIL, 2019)]",[]
AD367613,False,0,False,[],[]
AD367610,True,2,False,"[(NO, , , ), (NO, , , )]",[]
AD367697,False,0,False,[],[]


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [96]:
backgroundByCase['AD301211'] = getBackgroundRegister(case, registersByPatient.get_group(pId))

In [97]:
registros[registros.NumeroHistoria == 216835]

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1361304,216835,AD366773,1,OBSTETRICA,2020-01-07 08:55:48.203,NaN,NaN,RFLOREZ,72133854,20190409,...,False,NaN,NaN,NaN,2.0,NaN,NaT,NaN,False,NaN
1365508,216835,AD367822,20,Registro Evaluaci&#243;n Preanastesica,2020-01-15 12:40:02.610,NaN,NaN,GDELRIO,73084739,20190409,...,False,NaN,NaN,NaN,2.0,NaN,NaT,NaN,False,NaN
1366158,216835,AD367925,94,Notas de Ingreso a Piso,2020-01-16 15:24:04.307,NaN,NaN,NCARMONA,45470969,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2020-01-16 15:20:00,NaN,False,NaN
1366163,216835,AD367925,165,Registro del recién nacido,2020-01-16 15:33:44.960,NaN,Padre,sa,1047467767,20190409,...,False,NaN,NaN,NaN,3.0,NaN,NaT,NaN,False,NaN
1366194,216835,AD367925,145,NaN,2020-01-16 16:59:59.857,NaN,NaN,RDIAZ,18938424,20190409,...,False,NaN,NaN,NaN,3.0,NaN,NaT,NaN,False,NaN
1366262,216835,AD367925,94,Notas de Ingreso a Piso,2020-01-16 21:52:40.170,NaN,NaN,OSALASG,72490114,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2020-01-16 21:46:00,NaN,False,NaN
1366420,216835,AD367925,106,Evoluci&#243;nMA&#209;ANA,2020-01-17 09:56:41.100,94.0,1366262,JRAMIREZ,9068204,20190409,...,True,NaN,NaN,NaN,1.0,NaN,2020-01-17 09:49:00,NaN,False,NaN
1366423,216835,AD367925,46,Registro de Incapacidad,2020-01-17 10:00:22.400,NaN,NaN,JRAMIREZ,9068204,20190409,...,False,NaN,NaN,NaN,1.0,NaN,NaT,NaN,False,NaN
1366425,216835,AD367925,450,Epicrisis de la Historia Notas de Ingreso a Piso,2020-01-17 10:01:49.257,1.0,AD367925,JRAMIREZ,9068204,20190409,...,False,NaN,NaN,NaN,1.0,NaN,NaT,NaN,False,NaN


In [101]:
registros[registros.NumeroHistoria == 167552].DiagnosticoRelacionado1

Id
818364     N912
1011748    N872
1033395    N872
1046112    N872
1048307    N872
1088621     NaN
1088659     NaN
1088660    N872
1100597    Z488
1151993    N872
1177067    N870
1199029    N761
1263247    N761
Name: Diagnostico, dtype: object

In [105]:
pacientes.loc[167552]

Identificacion           VEN12758449
Carnet                           NaN
Nom1Afil                      TERESA
Nom2Afil                         NaN
Ape1Afil                   SAGOSTACE
                            ...     
ParentescoResponsable              1
DirResponsable                   NaN
Creencia                         NaN
NombreResponsable                NaN
EstadoEmbarazo                   NaN
Name: 167552, Length: 86, dtype: object

In [104]:
#Epicrisis / Ingreso de urgencias
prettyPrintXML(registros.loc[1088659].RegistroXML)

<?xml version="1.0" ?>
<C Asunto="" Caso="AD301211" CentroA="01" Diagnostico="NULL" IdPaciente="167552" Modulo="03" Padre="" PlanAdm="100" Prestador="22433520" Raiz="" Registro="145">
	<row NombreCampo="IdDescripcion" NombreTabla="CamposEntero" ValorCampo="46542"/>
	<row NombreCampo="CodigoActo" NombreTabla="CamposTexto" ValorCampo="CP63225"/>
	<row NombreCampo="IdPaciente" NombreTabla="CamposEntero" ValorCampo="167552"/>
	<row NombreCampo="CirugiaProgramada" NombreTabla="CamposTexto" ValorCampo="01:00:00 02:00:00 - INGRID MARIA STRUSS GARCIA"/>
	<row NombreCampo="Consulta" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="SaturacionOxigeno" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="Glucometria" NombreTabla="CamposTexto" ValorCampo=""/>
</C>



In [42]:
prettyPrintXML(registros.loc[1369859, 'RegistroXML'])

<?xml version="1.0" ?>
<C Asunto="Ingreso de Urgencias" Caso="AD368860" CentroA="01" CodigoDiagnosticoRelacionado1="" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="R102" IdPaciente="209748" Modulo="03" Padre="NULL" PlanAdm="1122018" Prestador="8865876" Raiz="NULL" Registro="07">
	<row NombreCampo="TipoConsulta" NombreTabla="CamposTexto" ValorCampo="OBSTETRICA"/>
	<row NombreCampo="PlanAdministradora" NombreTabla="CamposTexto" ValorCampo="ASOCIACION MUTUAL SER / ASOCIACION MUTUAL SER"/>
	<row NombreCampo="MotivoConsulta" NombreTabla="CamposTexto" ValorCampo="&amp;quot;ESTOY BOTANDO LIQUIDOS Y TENGO DOLORES&amp;quot;
"/>
	<row NombreCampo="EnfermedadActual" NombreTabla="CamposTexto" ValorCampo="PACIENTE DE 23 A&amp;#209;OS DE EDAD QUE CONSULTA POR UN CUADRO CLINICO DE 2 HORAS DE EVOLUCION CONSISTENTE EN DOLOR TIPO CONTRACCION EN HIPOGASTRIO QUE SE IRRADIA A REGION LUMBOSACRA, ACOMPA&amp;#209;ADO DE SALIDA DE LIQUIDOS POR GENITALES. NO ESTA ASOCIADO A OTRA

In [28]:
print(errorsEpicrisis)

28


In [ ]:
#TODO: check that operation and diagnosis are compatible.

In [ ]:
getprocID(i.XmlDescripcion)
for k,c in interventionToCase[k].iterrows():
    prettyPrintXML(c.RegistroXML)

In [ ]:
d = getEpicrisis('AD152124', casos)

In [ ]:
import xml.etree.ElementTree as ET


In [ ]:
def parseGestationalAge(s):
    pass

In [ ]:
import tqdm
casos_wo_epi = 0
casos_several_epi = 0
results ={}
for i,c in tqdm.tqdm_notebook(enumerate(registros.groupby('Caso')), total = len(casos)):
    if i == 1000:
        break
    epicrisis= getEpicrisis(c[0], c[1])
    if len(epicrisis) == 0:    
        casos_wo_epi += 1
    elif len(epicrisis) >= 2:    
        casos_several_epi += 1
    else:
        stringRegistro = epicrisis.RegistroXML.iloc[0]
        results[c[0]] = parseGPCA_and_fum(stringRegistro)
        data = {}
        for cc in c[1].iterrows():
            data = getDataFromCase(cc[1])
            if data:
                break
        results[c[0]].update(data)
    #I am not looking at echo information
    #Search also for "ultimo parto"
    #Search for height, weight and vital signs (can they change?) in the rest of the report. They are a row field in HTML.

In [ ]:
print('With epi: %d' % len(results))
print('Missing epi: %d' % casos_wo_epi)
print('Several epi: %d' % casos_several_epi)

In [ ]:
pd = pandas.DataFrame.from_dict(results, orient = 'index')
np.sum(pd.fum_OK != 0), np.sum(pd.fum), np.sum(pd.GPCA_OK), len(pd) 

In [ ]:
pd[pd.fum_OK == False]

In [ ]:
epicrisis= getEpicrisis('AD321568', registros)
stringRegistro = epicrisis.RegistroXML.iloc[0]
parseGPCA_and_fum(stringRegistro)

In [ ]:
allRegisters = registros[registros.Caso == 'AD257917']
epicrisis= getEpicrisis('AD257917', registros[registros.Caso == 'AD257917'])
s = epicrisis.RegistroXML.iloc[0]
l = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML",ET.fromstring(s).findall('row')))[0]
text = pars.unescape(l.attrib['ValorCampo'])
text

In [ ]:
prettyPrintXML(s) 

In [ ]:
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGCPCA = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGCPCA += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGCPCA += '|'
patternGCPCA += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGCPCA += '|' # C and A can be missing, or in other order
patternGCPCA += ')'
searchGCPCA = re.compile(patternGCPCA, flags=0)
searchGCPCA.findall(epicrisis.iloc[0].RegistroXML)

In [ ]:
import html
xmlString = epicrisis.iloc[0].RegistroXML

r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML",ET.fromstring(xmlString).findall('row')))
text = pars.unescape(r[0].attrib['ValorCampo'])
line = re.findall('^.*G' + sep + '[0-9]+.*$', text, re.M)
if len(line) != 2:
    #return False
    pass
G = re.findall('G' + sep + '([0-9]+)', line[0])[0]
A = re.findall('A' + sep + '([0-9]+)', line[0])[0]
C = re.findall('C' + sep + '([0-9]+)', line[0])[0]
P = re.findall('P' + sep + '([0-9]+)', line[0])[0] 

In [ ]:
G

In [ ]:
text.attrib['ValorCampo']

In [ ]:
print(epicrisis.iloc[0].RegistroXML)
searchGCPCA.findall(epicrisis.iloc[0].RegistroXML)

In [ ]:
for a in allRegisters.iterrows():
    print(prettyPrintXML(a[1].RegistroXML))

In [ ]:
r = registros.loc[393965]

r = xml.dom.minidom.parseString(stringRegistro) #r.RegistroXML)
print(r.toprettyxml())

In [ ]:
for c in r.childNodes:
    print(c)
    for cc in c.childNodes:
        print(cc, cc.attributes['NombreCampo'].value)

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string

In [ ]:
rString = registros.RegistroXML[465351]
register = ET.fromstring(rString)

In [ ]:
r = xml.dom.minidom.parseString(rString)
print(r.toprettyxml())

In [ ]:
i = 57
print(i, v[i].attributes['NombreCampo'].value,v[i].nodeValue)
print (" ".join(t.nodeValue for t in v[i].childNodes if t.nodeType == t.TEXT_NODE))

In [ ]:
from html.parser import HTMLParser
pars =HTMLParser()
pars.unescape(s)